In [1]:
'''
掛載Google硬碟
安裝套件
引用套件
APP應用準備
消息準備
handler執行方法設計
啟動應用

'''

'\n掛載Google硬碟\n安裝套件\n引用套件\nAPP應用準備\n消息準備\nhandler執行方法設計\n啟動應用\n\n'

In [2]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
'''

流程解析

'''


'\n\n流程解析\n\n'

In [4]:
!pip install line-bot-sdk flask flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 24.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


In [5]:
'''
引用套件
'''

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [6]:
'''
建置主程序

建置handler與 line_bot_api
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
# 跟line 溝通用
line_bot_api = LineBotApi("vtaRo4pkLqIZjk50gBJ53fFtMpg+8unl4TvLKbyvENmbWkzdriMNIeVm87ZWRBdTeDL7YM4hGte+BrNMmYscJ1WNPvCQbheXUbVevrlKB3zP3CaeRjbPQRPHMsqW+037bLOogqjOrh5ssRj53kqQMQdB04t89/1O/w1cDnyilFU=")
# 收發訊息用
handler = WebhookHandler("de385af73d2121e65ca6fc0490991b02")

In [7]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [13]:
'''
Button篇
    設定模板消息，指定其參數細節。

'''


#引入所需要的消息與模板消息
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)

#引入按鍵模板
from linebot.models.template import(
    ButtonsTemplate
)


'''
alt_text: Line簡覽視窗所出現的說明文字
template: 所使用的模板
ButtonsTemplate: 按鍵模板
    thumbnail_image_url: 展示圖片
    title: 標題
    text: 說明文字
    actions: 模板行為所使用的行為
    data: 觸發postback後用戶回傳值，可以對其做商業邏輯處理

'''
# 其他TemplateSendMessage
# Python line-bot-sdk ConfirmTemplate 關鍵字
# 最多四種ButtonsTemplate
buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        title='更多幫助',
        text='請點擊下方按鈕獲得更多幫助',
        actions=[
          {
            "type": "postback",
            "label": "企業，查找商業結合方案",
            "text": "[::text:]尋找BD",
            "data": "Data1"
          },
          {
            "type": "postback",
            "label": "開發者尋求教學",
            "uri": "[::text:]求助專家",
            "data": "Data2"
          },
          {
            "type": "uri",
            "label": "url scheme",
            "uri": "https://line.me/R/nv/camera/",
          },
          {
            "type": "uri",
            "label": "打電話",
            "uri": "tel://3345678",

          },
        ],
  )
)

In [14]:
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
  "@more":buttons_template_message,
}

In [15]:
# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    if(event.message.text.find('@')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="字典內查無此字，請輸入@more")
        )

In [16]:
#用戶點擊button後，觸發postback event，對其回傳做相對應處理

@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    if (event.postback.data.find('Data1')== 0):
        with open("user_profile_business.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='請稍待，會有專人與您聯繫'
                )
            )
    elif (event.postback.data.find('Data2') == 0):
        with open("user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='請稍待，我們會派專家與您聯繫'
                )
            )
    else:
        pass

In [17]:
# 主程序運行
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://4420-35-224-241-194.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{"destination":"U54dcf9ab5f592860405fcec0d6979997","events":[{"type":"message","message":{"type":"text","id":"17810981325727","text":"@more"},"webhookEventId":"01GVMFE0PT1JNY6WXRKRPE0J21","deliveryContext":{"isRedelivery":false},"timestamp":1678945747246,"source":{"type":"user","userId":"Ua19163cfb927dce013b280948e91caf0"},"replyToken":"63cf8bc54d8140f98c42fdc0e2da1b10","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 05:49:09] "POST / HTTP/1.1" 200 -


{"destination":"U54dcf9ab5f592860405fcec0d6979997","events":[{"type":"postback","postback":{"data":"Data2"},"webhookEventId":"01GVMFEKATNGFSEZJVG7EXR90W","deliveryContext":{"isRedelivery":false},"timestamp":1678945766694,"source":{"type":"user","userId":"Ua19163cfb927dce013b280948e91caf0"},"replyToken":"d5eb5cf29e0c42debed3d3e5f8e096dd","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 05:49:27] "POST / HTTP/1.1" 200 -


{"destination":"U54dcf9ab5f592860405fcec0d6979997","events":[{"type":"postback","postback":{"data":"Data1"},"webhookEventId":"01GVMFEPWFCT31XT2H9R3JS50D","deliveryContext":{"isRedelivery":false},"timestamp":1678945769880,"source":{"type":"user","userId":"Ua19163cfb927dce013b280948e91caf0"},"replyToken":"fa1d64935d184550ae6e255e9b27c282","mode":"active"},{"type":"message","message":{"type":"text","id":"17810983026491","text":"[::text:]尋找BD"},"webhookEventId":"01GVMFEPWG7P8DRBDFSV414K3J","deliveryContext":{"isRedelivery":false},"timestamp":1678945769881,"source":{"type":"user","userId":"Ua19163cfb927dce013b280948e91caf0"},"replyToken":"2e30b9bc2c42442fa1403efdb8895ef1","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 05:49:31] "POST / HTTP/1.1" 200 -
